### Importing required libraries

In [14]:
import pandas as pd
from sklearn.utils import shuffle
import numpy as np
from nltk.corpus import stopwords
import string
from collections import Counter
from string import punctuation
import os

### Downloading english stopwords defined in nltk module

In [15]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\amand\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
#Give path of new_labels.txt with /
pd.read_csv(r'C:\Users\amand\OneDrive\Desktop\tmp2\new_labels.txt')


,file,label
0,A00-1001.txt,Speech
1,A00-1002.txt,Translation
2,A00-1004.txt,Translation
3,A00-1005.txt,Speech
4,A00-1006.txt,Translation
...,...,...
438,J96-3002.txt,Grammar
439,M93-1003.txt,IE
440,M93-1004.txt,IE
441,M93-1006.txt,IE


In [18]:
#Reading file names and labels

file_labels = pd.read_csv(r'C:\Users\amand\OneDrive\Desktop\tmp2\new_labels.txt')
file_labels.to_csv (r'C:\Users\amand\OneDrive\Desktop\tmp2\read_label.csv')

In [19]:
files = file_labels.file

In [20]:
#Vieing label distribution
file_labels.label.value_counts()

Speech           121
Translation       93
IE                67
Grammar           55
Lexical           35
QA                27
Summarization     23
NE                22
Name: label, dtype: int64

# Extract top words in the data

In [21]:
# turn a doc into clean tokens
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return tokens

# load doc and add to vocab
def add_doc_to_vocab(filename, vocab):
    # load doc
    with open(filename,'r') as fd:
        lines = fd.read()
        #transforming data to lower 
        lines = lines.lower()
        lines_list = lines.split()
        #some files doesn't contain abstract and it thros error so to eliminate it we are using try cath
        try:
            lines = lines_list[1:lines_list.index('introduction')-1]
            if len(lines)<10:
                lines = lines_list[0:250]
        except:
            lines = lines_list[0:250]
        lines = ' '.join([str(elem) for elem in lines]) 
        tokens = clean_doc(lines)
        # update counts
        vocab.update(tokens)
    
    
def save_list(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

# define vocab
vocab = Counter()

# add all docs to vocab

for file in files:
    #Need to give path of cleaned_acl_arc with /
    open_file = r'C:\Users\amand\OneDrive\Desktop\text mining project\corpus\cleaned_acl_arc/' + file
    print(file)
    add_doc_to_vocab(open_file, vocab)
        


# print the size of the vocab
print(len(vocab))

# print the top words in the vocab
print(vocab.most_common(200))

min_occurance = 5
tokens = [k for k,c in vocab.items() if c >= min_occurance]
print(len(tokens))

# save tokens to a vocabulary file
save_list(tokens, r'C:\Users\amand\OneDrive\Desktop\tmp2\vocabulary.txt')

A00-1001.txt
A00-1002.txt
A00-1004.txt
A00-1005.txt
A00-1006.txt
A00-1007.txt
A00-1009.txt
A00-1010.txt
A00-1011.txt
A00-1012.txt
A00-1013.txt
A00-1015.txt
A00-1014.txt
A00-1016.txt
A00-1018.txt
A00-1021.txt
A00-1023.txt
A00-1023.txt
A00-1025.txt
A00-1027.txt
A00-1028.txt
A00-1029.txt
A00-1032.txt
A00-1034.txt
A00-1035.txt
A00-1036.txt
A00-1038.txt
A00-1039.txt
A00-1040.txt
A00-1041.txt
A00-1042.txt
A00-1043.txt
A00-1044.txt
A00-1045.txt
A00-2001.txt
A00-2011.txt
A00-2013.txt
A00-2014.txt
A00-2019.txt
A00-2021.txt
A00-2022.txt
A00-2024.txt
A00-2025.txt
A00-2025.txt
A00-2027.txt
A00-2028.txt
A00-2029.txt
A00-2030.txt
A00-2033.txt
A00-2035.txt
A00-2036.txt
A00-2037.txt
A00-2039.txt
A00-3004.txt
A00-3007.txt
A83-1005.txt
A83-1009.txt
A83-1017.txt
A83-1019.txt
A88-1010.txt
A88-1014.txt
A88-1016.txt
A88-1018.txt
A88-1020.txt
A94-1005.txt
A94-1006.txt
A94-1007.txt
A94-1008.txt
A94-1009.txt
A94-1015.txt
A94-1016.txt
A94-1017.txt
A94-1019.txt
A94-1044.txt
A94-1047.txt
A97-1001.txt
A97-1003.txt

# Convert entire data into data ready to feed to model

In [22]:
# load vocabulary
vocab_filename = r'C:\Users\amand\OneDrive\Desktop\tmp2\vocabulary.txt'
file = open(vocab_filename, 'r',errors='ignore')
text = file.read()
vocab = text.split()
vocab = set(vocab)

In [23]:

# load doc into memory
def load_doc(filename):
    # open the file as read only
    txt_file = open(filename, 'r',errors='ignore')
    # read all text
    text = txt_file.read()
    text = text.lower()
    text_list = text.split()
    
    try:
        text = text_list[1:text_list.index('introduction')-1]
        if len(text)<10:
                text = text_list[0:250]
    except:
        text = text_list[0:250]
    text = ' '.join([str(elem) for elem in text]) 
    # close the file
    txt_file.close()
    return text

# load doc, clean and return line of tokens
# turn a doc into clean tokens
def clean_doc(doc):
    # split into tokens by white space
    rtr = []
    split_text = doc.split()
    for tokens in split_text:
        tokens = tokens.split()
        # remove punctuation from each token
        table = str.maketrans('', '', punctuation)
        tokens = [w.translate(table) for w in tokens]
        # remove remaining tokens that are not alphabetic
        tokens = [word for word in tokens if word.isalpha()]
        # filter out stop words
        stop_words = set(stopwords.words('english'))
        tokens = [w for w in tokens if not w in stop_words]
        # filter out short tokens
        tokens = [word for word in tokens if len(word) > 1]
        if len(tokens) > 0:
            rtr.append(tokens)
    return rtr

def save_list(lines, filename):
    file = open(filename, 'w')
    for line in lines:
        data = ' '.join(line)
        if len(data) >=1 :
            file.write(data)
            file.write('\n')
    file.close()
    
    
def doc_to_line(filename,file, vocab):
    # load the doc
    doc = load_doc(filename+file)
    final = []
    # clean doc
    tokens = clean_doc(doc)
    if len(tokens) == 0:
        print(file)
    # filter by vocab
    for i in range (len(tokens)):
        t = [w for w in tokens[i] if w in vocab]
        final.append(' '.join(t))
    final = list(filter(None, final)) 
    return final
         
 
# load vocabulary
vocab_filename = r'C:\Users\amand\OneDrive\Desktop\tmp2\vocabulary.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)

nlp = []
for file in files:
    #Need to give path of cleaned_acl_arc with /
    nlp.append(doc_to_line(r'C:\Users\amand\OneDrive\Desktop\text mining project\corpus\cleaned_acl_arc/' ,file,vocab))

save_list(nlp, r'C:\Users\amand\OneDrive\Desktop\tmp2\finished.txt')